In [1]:
# Task - 1, when n = 8
from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName('MyApp')
sc = SparkContext(conf=conf)


In [2]:
A = sc.textFile('duom_cut.txt')


In [3]:
# map

def map_func(x):
    key = ''
    value = float(0)
    for entry in x:
        e_key, e_value = entry.split('=')
        if e_key == 'svorio grupe':
            key = e_value.replace('}}', '').replace('{{', '')
        elif e_key == 'svoris':
            try:
                value = float(e_value.replace('}}', '').replace('{{', ''))
            except Exception as e:
                pass
    return (key, value)


y = A.map(lambda x: x.strip())\
    .flatMap(lambda x: x.split('}}{{'))\
    .map(lambda x: x[2:] if x.startswith('{{') else x)\
    .map(lambda x: x[:-2] if x.startswith('}}') else x)\
    .map(lambda x: x.split('}{'))\
    .map(map_func)\
    .filter(lambda x: x[0] != '')

print(y.take(20))


[('<300', 76.0), ('<300', 81.5), ('<50', 45.0), ('<50', 43.0), ('<50', 40.0), ('<50', 41.0), ('<300', 104.0), ('<300', 85.15), ('<300', 60.0), ('<300', 58.0), ('<300', 84.0), ('<50', 3.0), ('<300', 119.45), ('<300', 105.9), ('<50', 0.0), ('<300', 84.0), ('>300', 764.2), ('<300', 113.05), ('<300', 70.0), ('<50', 37.3)]


In [4]:
def reduce_func(x, y):
    return (x[0] + y[0], x[1] + y[1])


reduced = y.mapValues(lambda x:  (x, 1))\
    .reduceByKey(reduce_func)

print(reduced.take(20))

[('>300', (10852.599999999999, 21)), ('<300', (17646.5, 176)), ('<50', (2959.2600000000007, 285))]


In [7]:
# Statistics

avg = reduced.mapValues(lambda x: x[0] / x[1])
print(avg.collect())

min = reduced.mapValues(lambda x: x[0] if x[0] < x[1] else x[1])
print(min.collect())

max = reduced.mapValues(lambda x: x[0] if x[0] > x[1] else x[1])
print(max.collect())


[('>300', 516.7904761904762), ('<300', 100.26420454545455), ('<50', 10.383368421052634)]
[('>300', 21), ('<300', 176), ('<50', 285)]
[('>300', 10852.599999999999), ('<300', 17646.5), ('<50', 2959.2600000000007)]
